In [1]:
import os
import xarray as xa
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib qt5

# navigate to folder that contains subfolders of dates '20240527', '20240528', ..., '20240621'
# subfolders of dates contain CINDI-3 submission files
os.chdir('submission_data/v1.2')

In [2]:
# Limits (as fraction of hour) allowed allowed around start time
lower_threshold = 3/120     # 3/120                 # start time before hh:mm:30 of actual start time   (Karin's definition: start time - 1 minute)
upper_threshold = 3/120     # 3/120                 # stop time after hh:mm:30 of actual start time     (Karin's definition: start time + 1.999 minute)
lower_threshold_zenith = 3/120     # 3/120          # start time before hh:mm:30 of actual start time   (Karin's definition: start time - 1 minute)
upper_threshold_zenith = 3/120     # 3/120          # stop time after hh:mm:30 of actual start time     (Karin's definition: start time + 1.999 minute)

dates = os.listdir()[6:-2]      # all dates except '20240527','20240528','20240529','20240530','20240531','20240601','20240620','20240621'
data = xa.Dataset(
    data_vars=dict(
        DSCD=(['PAN_ID','DATE','REF','GAS','TIME','VEA'],np.zeros((3,len(dates),3,13,500,10))*np.nan),
        timestamp=(['PAN_ID','DATE','REF','GAS','TIME','VEA'],np.zeros((3,len(dates),3,13,500,10))*np.nan),
        ),
    coords=dict(PAN_ID=('PAN_ID',np.array(['34','35','36'])), DATE=('DATE', np.array(dates)), REF=('REF',np.array(['DAILYREF', 'SEQREF','FIXREF'])), GAS=('GAS',np.array(['O4VIS','O4UV','O3VIS','O3UV','NO2VIS-SMALL','NO2VIS','NO2UV','HONO','HCHO-WIDE','HCHO','CHOCHO','BrO','BRO'])), TIME=('TIME', np.zeros((500))), VEA=('VEA', np.array([-2,1,2,3,4,6,8,15,30,90]))),
)
data_timeflagged = xa.Dataset(
    data_vars=dict(
        DSCD=(['PAN_ID','DATE','REF','GAS','TIME','VEA'],np.zeros((3,len(dates),3,13,500,10))*np.nan),
        timestamp=(['PAN_ID','DATE','REF','GAS','TIME','VEA'],np.zeros((3,len(dates),3,13,500,10))*np.nan),
        ),
    coords=dict(PAN_ID=('PAN_ID',np.array(['34','35','36'])), DATE=('DATE', np.array(dates)), REF=('REF',np.array(['DAILYREF', 'SEQREF','FIXREF'])), GAS=('GAS',np.array(['O4VIS','O4UV','O3VIS','O3UV','NO2VIS-SMALL','NO2VIS','NO2UV','HONO','HCHO-WIDE','HCHO','CHOCHO','BrO','BRO'])), TIME=('TIME', np.zeros((500))), VEA=('VEA', np.array([-2,1,2,3,4,6,8,15,30,90]))),
)

# center time (=30s after start time) of off-axis meas angles as fraction of hour according to CINDI-3 schedule
time_offaxis_schedule = dict(zip([-2, 1, 2, 3, 4, 6, 8, 15, 30, 90]+[-2, 1, 2, 3, 4, 6, 8, 15, 30, 90],
                      list(np.vstack([np.arange(21/120,40/120,1/60),np.arange(81/120,100/120,1/60)]).T)))

# center time (=30s after start time) of Zenith meas as fraction of hour according to CINDI-3 schedule, in hourly schedule
time_zenith_schedule = np.array([11/120,13/120,15/120,17/120,19/120,39/120,71/120,73/120,75/120,77/120,79/120,99/120])
# center time (=30s after start time) of Zenith meas as fraction of hour according to CINDI-3 schedule, in noon schedule
time_zenith_schedule_noon = np.array([11/120,13/120,15/120,17/120,19/120,39/120,99/120])

for date in dates:
    os.chdir(date)
    for file in os.listdir():
        p_id = file.split('_')[2]
        ref = file.split('_')[4]
        gas = file.split('_')[3]
        with open(file,'r') as f:
            elev_angles_list = []
            azim_angles_list = []
            data_list = []
            time_list = []
            lines = f.readlines()
            for line in lines:
                if '% ' in line[:2]:
                    if gas in line[2:] and 'Col' in line[2:] and 'Error' not in line:
                        gasindex = int(line.split('Col ')[-1].split(':')[0]) - 1
                        gasentry = line.split(': ')[1]
                else:
                    elev_angles_list.append(float(line.split(' ')[5]))
                    azim_angles_list.append(float(line.split(' ')[6]))
                    data_list.append(float(line.split(' ')[gasindex]))
                    time_list.append(np.float64(line.split(' ')[1]))
        time_angles_meas = dict(zip(elev_angles_list,time_list))
        angles_stored = []
        flag_zenith_hourly=False
        flag_zenith_noon=False
        for i_angle, angle in enumerate(elev_angles_list):
            if angle in data.VEA.data and azim_angles_list[i_angle] == 287 and angle not in angles_stored:
                mask_azim_elev = np.multiply(np.array(elev_angles_list)==angle,np.array(azim_angles_list)==287)
                data.DSCD.sel(PAN_ID=p_id,DATE=date,REF=ref,GAS=gas,VEA=angle).isel(TIME=slice(0,len(np.array(data_list)[mask_azim_elev])))[:] = np.array(data_list)[mask_azim_elev]
                data.timestamp.sel(PAN_ID=p_id,DATE=date,REF=ref,GAS=gas,VEA=angle).isel(TIME=slice(0,len(np.array(data_list)[mask_azim_elev])))[:] = np.array(time_list)[mask_azim_elev]
                if angle == 90:                                                                 # Zenith
                    if (11+1/6) < time_list[i_angle] < (12+1/6) and flag_zenith_noon==False:    # noon schedule Zenith
                        mask_list = []
                        for a in time_zenith_schedule_noon:
                            mask_list.append((a - lower_threshold_zenith < (time_list-np.floor(time_list))) & ((time_list-np.floor(time_list)) < a + upper_threshold_zenith))

                        mask_zen_noon = mask_list[0]
                        for sublist in mask_list[1:]:
                            mask_zen_noon = np.logical_or(mask_zen_noon, sublist)
                        flag_zenith_noon=True

                        print('without timeflag %s Pandora %s %s %s: %s entries at VEA=%s° (hourly)' % (date, p_id, gas, ref, len(np.array(data_list)[mask_azim_elev]), angle))
                    elif flag_zenith_hourly == False:                                           # hourly schedule Zenith
                        mask_list = []
                        for a in time_zenith_schedule:
                            mask_list.append((a - lower_threshold_zenith < (time_list-np.floor(time_list))) & ((time_list-np.floor(time_list)) < a + upper_threshold_zenith) & (np.array(time_list) > 4)  & (np.array(time_list) < 20))

                        mask_zen_hourly = mask_list[0]
                        for sublist in mask_list[1:]:
                            mask_zen_hourly = np.logical_or(mask_zen_hourly, sublist)
                        flag_zenith_hourly=True

                        print('without timeflag %s Pandora %s %s %s: %s entries at VEA=%s° (noon)' % (date, p_id, gas, ref, len(np.array(data_list)[mask_azim_elev]), angle))
                else:                                                                           # off-axis (hourly schedule)
                    print('without timeflag %s Pandora %s %s %s: %s entries at VEA=%s°' % (date, p_id, gas, ref, len(np.array(data_list)[mask_azim_elev]), angle))
                    angles_stored.append(angle)
                    mask_time_angles = np.logical_or(
                            np.multiply(
                            list(time_offaxis_schedule[angle][0] - lower_threshold < (time_list - np.floor(time_list))), 
                            list((time_list - np.floor(time_list)) < time_offaxis_schedule[angle][0] + upper_threshold)
                            ),
                            np.multiply(
                            list(time_offaxis_schedule[angle][1] - lower_threshold < (time_list - np.floor(time_list))), 
                            list((time_list - np.floor(time_list)) < time_offaxis_schedule[angle][1] + upper_threshold)
                            )
                        )
                if angle == 90 and (flag_zenith_hourly*flag_zenith_noon):
                    mask_time_angles = mask_zen_noon+mask_zen_hourly
                    angles_stored.append(angle)
                if (angle != 90) != (flag_zenith_hourly*flag_zenith_noon):
                    print('with timeflag %s Pandora %s %s %s: %s entries at VEA=%s°' % (date, p_id, gas, ref, len(np.array(data_list)[np.multiply(mask_time_angles,mask_azim_elev)]), angle))
                    data_timeflagged.DSCD.sel(PAN_ID=p_id,DATE=date,REF=ref,GAS=gas,VEA=angle).isel(TIME=slice(0,len(np.array(data_list)[np.multiply(mask_time_angles,mask_azim_elev)])))[:] = np.array(data_list)[np.multiply(mask_time_angles,mask_azim_elev)]
                    data_timeflagged.timestamp.sel(PAN_ID=p_id,DATE=date,REF=ref,GAS=gas,VEA=angle).isel(TIME=slice(0,len(np.array(time_list)[np.multiply(mask_time_angles,mask_azim_elev)])))[:] = np.array(time_list)[np.multiply(mask_time_angles,mask_azim_elev)]

    os.chdir('..')

without timeflag 20240602 Pandora 34 BrO DAILYREF: 124 entries at VEA=90.0° (noon)
without timeflag 20240602 Pandora 34 BrO DAILYREF: 28 entries at VEA=-2.0°
with timeflag 20240602 Pandora 34 BrO DAILYREF: 28 entries at VEA=-2.0°
without timeflag 20240602 Pandora 34 BrO DAILYREF: 28 entries at VEA=1.0°
with timeflag 20240602 Pandora 34 BrO DAILYREF: 28 entries at VEA=1.0°
without timeflag 20240602 Pandora 34 BrO DAILYREF: 45 entries at VEA=2.0°
with timeflag 20240602 Pandora 34 BrO DAILYREF: 28 entries at VEA=2.0°
without timeflag 20240602 Pandora 34 BrO DAILYREF: 28 entries at VEA=3.0°
with timeflag 20240602 Pandora 34 BrO DAILYREF: 28 entries at VEA=3.0°
without timeflag 20240602 Pandora 34 BrO DAILYREF: 28 entries at VEA=4.0°
with timeflag 20240602 Pandora 34 BrO DAILYREF: 28 entries at VEA=4.0°
without timeflag 20240602 Pandora 34 BrO DAILYREF: 28 entries at VEA=6.0°
with timeflag 20240602 Pandora 34 BrO DAILYREF: 28 entries at VEA=6.0°
without timeflag 20240602 Pandora 34 BrO DAIL

In [3]:
# specify date to be plotted in subfigure "DSCD_over_minutes" (if date='', all dates are plotted)
date='20240610'

# plot all data (not filtered by time limits)?
plot_unflagged=False

In [4]:
import warnings
warnings.filterwarnings("ignore")

# colors of bars same as in CINDI-3 report
refcolors=['grey','red','blue']

# change labels in subplots from YYYMMDD to DD.MM.
date_labels = []
for d in data.DATE.data:
    date_labels.append(d[-2:]+'.'+d[-4:-2]+'.')

# for p_id in ['36']:
for p_id in data.PAN_ID.data:
    print('Pandora '+p_id)
    # for gas in ['O4VIS']:
    for gas in data.GAS.data:
        if plot_unflagged:
            print('     '+gas)
            fig1, ax1 = plt.subplots(3,5, figsize=(10, 6), gridspec_kw={'height_ratios': [1, 1, 0.5]})
            fig1.canvas.manager.set_window_title(p_id+'_'+gas)
            
            for axx in ax1[2, :]:
                fig1.delaxes(axx)

            bottom_ax1 = fig1.add_subplot(3, 1, 3)

            c=0
            width=0.8
            for ref in data.REF.data:
                x=0
                y=0
                for vea in data.VEA.data:
                    size=10
                    ax1[x,y].bar(data.sel(PAN_ID=p_id,REF=ref,GAS=gas,VEA=vea).DATE.data,
                        data.sel(PAN_ID=p_id,REF=ref,GAS=gas,VEA=vea).DSCD.count(dim='TIME'),color=refcolors[c],width=width)
                    ax1[x,y].set_title('VEA=%s°' % (vea))
                    ax1[x,y].set_xticklabels(date_labels, rotation=45)
                    ax1[x,y].set_ylabel('#')
                    ax1[x,y].grid()
                    if vea == 90:
                        ax1[x,y].set_ylim([0,180])
                    else:
                        ax1[x,y].set_ylim([0,31])
                    if ref == data.REF.data[0]:
                        if date != '':
                            bottom_ax1.scatter((data.timestamp.sel(PAN_ID=p_id, REF=ref, GAS=gas, VEA=vea, DATE=date).data - np.floor(data.timestamp.sel(PAN_ID=p_id, REF=ref, GAS=gas, VEA=vea, DATE=date).data))*60, data.DSCD.sel(PAN_ID=p_id, REF=ref, GAS=gas, VEA=vea, DATE=date).data, s=size, label='VEA=%s°'%vea)
                        else:
                            bottom_ax1.scatter((data.timestamp.sel(PAN_ID=p_id, REF=ref, GAS=gas, VEA=vea).data - np.floor(data.timestamp.sel(PAN_ID=p_id, REF=ref, GAS=gas, VEA=vea).data))*60, data.DSCD.sel(PAN_ID=p_id, REF=ref, GAS=gas, VEA=vea).data, s=size, label='VEA=%s°'%vea)
                        if vea == 90:
                            ax1[x,y].hlines(174,data.DATE.data[0],data.DATE.data[-1],'red')
                        else:
                            ax1[x,y].hlines(29,data.DATE.data[0],data.DATE.data[-1],'red')
                    else:
                        if date != '':
                            bottom_ax1.scatter((data.timestamp.sel(PAN_ID=p_id, REF=ref, GAS=gas, VEA=vea, DATE=date).data - np.floor(data.timestamp.sel(PAN_ID=p_id, REF=ref, GAS=gas, VEA=vea, DATE=date).data))*60, data.DSCD.sel(PAN_ID=p_id, REF=ref, GAS=gas, VEA=vea, DATE=date).data, s=size)
                        else:
                            bottom_ax1.scatter((data.timestamp.sel(PAN_ID=p_id, REF=ref, GAS=gas, VEA=vea).data - np.floor(data.timestamp.sel(PAN_ID=p_id, REF=ref, GAS=gas, VEA=vea).data))*60, data.DSCD.sel(PAN_ID=p_id, REF=ref, GAS=gas, VEA=vea).data, s=size)
                    if y==4:
                        y=0
                        x+=1
                    else:
                        y+=1

                plt.suptitle('%s, %s' % (p_id,gas))
                c+=1
                width-=0.3
        
            for i_vea in range(len(data.VEA.data)):
                bottom_ax1.axvspan((np.array(list(time_offaxis_schedule.values()))[i_vea,0]-lower_threshold)*60,(np.array(list(time_offaxis_schedule.values()))[i_vea,0]+upper_threshold)*60,-100,100,alpha=0.1,facecolor=list(mcolors.TABLEAU_COLORS.values())[i_vea])
                bottom_ax1.axvspan((np.array(list(time_offaxis_schedule.values()))[i_vea,1]-lower_threshold)*60,(np.array(list(time_offaxis_schedule.values()))[i_vea,1]+upper_threshold)*60,-100,100,alpha=0.1,facecolor=list(mcolors.TABLEAU_COLORS.values())[i_vea])
                if i_vea == len(data.VEA.data)-1:
                    bottom_ax1.vlines((np.array(list(time_offaxis_schedule.values()))[i_vea,0]-lower_threshold)*60,-5000,5000,colors=list(mcolors.TABLEAU_COLORS.values())[i_vea],linestyles='dotted',label='lower limit off-axis   (scheduled start time -%s)' % int((lower_threshold - 1/120)*60*60))
                    bottom_ax1.vlines((np.array(list(time_offaxis_schedule.values()))[i_vea,0]+upper_threshold)*60,-5000,5000,colors=list(mcolors.TABLEAU_COLORS.values())[i_vea],linestyles='dashed',label='upper limit off-axis   (scheduled start time +%s)' % int((upper_threshold + 1/120)*60*60))
                else:
                    bottom_ax1.vlines((np.array(list(time_offaxis_schedule.values()))[i_vea,0]-lower_threshold)*60,-5000,5000,colors=list(mcolors.TABLEAU_COLORS.values())[i_vea],linestyles='dotted')
                    bottom_ax1.vlines((np.array(list(time_offaxis_schedule.values()))[i_vea,0]+upper_threshold)*60,-5000,5000,colors=list(mcolors.TABLEAU_COLORS.values())[i_vea],linestyles='dashed')
                bottom_ax1.vlines((np.array(list(time_offaxis_schedule.values()))[i_vea,1]-lower_threshold)*60,-5000,5000,colors=list(mcolors.TABLEAU_COLORS.values())[i_vea],linestyles='dotted')
                bottom_ax1.vlines((np.array(list(time_offaxis_schedule.values()))[i_vea,1]+upper_threshold)*60,-5000,5000,colors=list(mcolors.TABLEAU_COLORS.values())[i_vea],linestyles='dashed')
            bottom_ax1.vlines((time_zenith_schedule[0]-lower_threshold_zenith)*60,-5000,5000,colors='grey',linestyles='dotted', label='lower limit Zenith   (scheduled start time -%s)' % int((lower_threshold_zenith - 1/120)*60*60))
            bottom_ax1.vlines((time_zenith_schedule[4]+upper_threshold_zenith)*60,-5000,5000,colors='grey',linestyles='dashed', label='upper limit Zenith   (scheduled start time +%s)' % int((upper_threshold_zenith + 1/120)*60*60))
            bottom_ax1.vlines((time_zenith_schedule[5]-lower_threshold_zenith)*60,-5000,5000,colors='grey',linestyles='dotted')
            bottom_ax1.vlines((time_zenith_schedule[5]+upper_threshold_zenith)*60,-5000,5000,colors='grey',linestyles='dashed')
            bottom_ax1.vlines((time_zenith_schedule[6]-lower_threshold_zenith)*60,-5000,5000,colors='grey',linestyles='dotted')
            bottom_ax1.vlines((time_zenith_schedule[-2]+upper_threshold_zenith)*60,-5000,5000,colors='grey',linestyles='dashed')
            bottom_ax1.vlines((time_zenith_schedule[-1]-lower_threshold_zenith)*60,-5000,5000,colors='grey',linestyles='dotted')
            bottom_ax1.vlines((time_zenith_schedule[-1]+upper_threshold_zenith)*60,-5000,5000,colors='grey',linestyles='dashed')
            bottom_ax1.grid()
            bottom_ax1.set_title('%s DSCD over minutes' % date)
            bottom_ax1.legend()
            bottom_ax1.set_xlabel('MINUTES')
            bottom_ax1.set_ylabel('DSCD')
            bottom_ax1.set_xlim([0,60])
            bottom_ax1.set_ylim([-100,100])
            bottom_ax1.set_xticks([0,5,10,15,20,25,30,35,40,45,50,55,60])
            plt.tight_layout()

        print('     timeflag '+gas)
        fig, ax = plt.subplots(3,5, figsize=(10, 6), gridspec_kw={'height_ratios': [1, 1, 0.5]})
        fig.canvas.manager.set_window_title(p_id+'_'+gas+'_timeflagged')

        for axx in ax[2, :]:
            fig.delaxes(axx)

        bottom_ax = fig.add_subplot(3, 1, 3)
        
        c=0
        width=0.8
        for ref in data.REF.data:
            x=0
            y=0
            for vea in data_timeflagged.VEA.data:
                size=10
                
                ax[x,y].bar(data_timeflagged.sel(PAN_ID=p_id,REF=ref,GAS=gas,VEA=vea).DATE.data,
                    data_timeflagged.sel(PAN_ID=p_id,REF=ref,GAS=gas,VEA=vea).DSCD.count(dim='TIME'),color=refcolors[c],width=width)
                ax[x,y].set_title('VEA=%s°' % (vea))
                ax[x,y].set_xticklabels(date_labels, rotation=45)
                ax[x,y].set_ylabel('#')
                ax[x,y].grid()
                if vea == 90:
                    ax[x,y].set_ylim([0,180])
                else:
                    ax[x,y].set_ylim([0,31])
                if ref == data.REF.data[0]:
                    if date != '':
                        bottom_ax.scatter((data_timeflagged.timestamp.sel(PAN_ID=p_id, REF=ref, GAS=gas, VEA=vea, DATE=date).data - np.floor(data_timeflagged.timestamp.sel(PAN_ID=p_id, REF=ref, GAS=gas, VEA=vea, DATE=date).data))*60, data_timeflagged.DSCD.sel(PAN_ID=p_id, REF=ref, GAS=gas, VEA=vea, DATE=date).data, s=size, label='VEA=%s°'%vea)
                    else:
                        bottom_ax.scatter((data_timeflagged.timestamp.sel(PAN_ID=p_id, REF=ref, GAS=gas, VEA=vea).data - np.floor(data_timeflagged.timestamp.sel(PAN_ID=p_id, REF=ref, GAS=gas, VEA=vea).data))*60, data_timeflagged.DSCD.sel(PAN_ID=p_id, REF=ref, GAS=gas, VEA=vea).data, s=size, label='VEA=%s°'%vea)
                    if vea == 90:
                        ax[x,y].hlines(174,data.DATE.data[0],data.DATE.data[-1],'red')
                    else:
                        ax[x,y].hlines(29,data.DATE.data[0],data.DATE.data[-1],'red')
                else:
                    if date != '':
                        bottom_ax.scatter((data_timeflagged.timestamp.sel(PAN_ID=p_id, REF=ref, GAS=gas, VEA=vea, DATE=date).data - np.floor(data_timeflagged.timestamp.sel(PAN_ID=p_id, REF=ref, GAS=gas, VEA=vea, DATE=date).data))*60, data_timeflagged.DSCD.sel(PAN_ID=p_id, REF=ref, GAS=gas, VEA=vea, DATE=date).data, s=size)
                    else:
                        bottom_ax.scatter((data_timeflagged.timestamp.sel(PAN_ID=p_id, REF=ref, GAS=gas, VEA=vea).data - np.floor(data_timeflagged.timestamp.sel(PAN_ID=p_id, REF=ref, GAS=gas, VEA=vea).data))*60, data_timeflagged.DSCD.sel(PAN_ID=p_id, REF=ref, GAS=gas, VEA=vea).data, s=size)
                if y==4:
                    y=0
                    x+=1
                else:
                    y+=1
            plt.suptitle('%s, %s, time flag' % (p_id,gas))
            c+=1
            width-=0.3
        for i_vea in range(len(data_timeflagged.VEA.data)):
            bottom_ax.axvspan((np.array(list(time_offaxis_schedule.values()))[i_vea,0]-lower_threshold_zenith)*60,(np.array(list(time_offaxis_schedule.values()))[i_vea,0]+upper_threshold)*60,-100,100,alpha=0.1,facecolor=list(mcolors.TABLEAU_COLORS.values())[i_vea])
            bottom_ax.axvspan((np.array(list(time_offaxis_schedule.values()))[i_vea,1]-lower_threshold_zenith)*60,(np.array(list(time_offaxis_schedule.values()))[i_vea,1]+upper_threshold)*60,-100,100,alpha=0.1,facecolor=list(mcolors.TABLEAU_COLORS.values())[i_vea])
            if i_vea == len(data_timeflagged.VEA.data)-1:
                bottom_ax.vlines((np.array(list(time_offaxis_schedule.values()))[i_vea,0]-lower_threshold)*60,-5000,5000,colors=list(mcolors.TABLEAU_COLORS.values())[i_vea],linestyles='dotted',label='lower limit off-axis    (scheduled start time -%s)' % int((lower_threshold - 1/120)*60*60))
                bottom_ax.vlines((np.array(list(time_offaxis_schedule.values()))[i_vea,0]+upper_threshold)*60,-5000,5000,colors=list(mcolors.TABLEAU_COLORS.values())[i_vea],linestyles='dashed',label='upper limit off-axis    (scheduled start time +%s)' % int((upper_threshold + 1/120)*60*60))
            else:
                bottom_ax.vlines((np.array(list(time_offaxis_schedule.values()))[i_vea,0]-lower_threshold)*60,-5000,5000,colors=list(mcolors.TABLEAU_COLORS.values())[i_vea],linestyles='dotted')
                bottom_ax.vlines((np.array(list(time_offaxis_schedule.values()))[i_vea,0]+upper_threshold)*60,-5000,5000,colors=list(mcolors.TABLEAU_COLORS.values())[i_vea],linestyles='dashed')
            bottom_ax.vlines((np.array(list(time_offaxis_schedule.values()))[i_vea,1]-lower_threshold)*60,-5000,5000,colors=list(mcolors.TABLEAU_COLORS.values())[i_vea],linestyles='dotted')
            bottom_ax.vlines((np.array(list(time_offaxis_schedule.values()))[i_vea,1]+upper_threshold)*60,-5000,5000,colors=list(mcolors.TABLEAU_COLORS.values())[i_vea],linestyles='dashed')
        bottom_ax.vlines((time_zenith_schedule[0]-lower_threshold_zenith)*60,-5000,5000,colors='grey',linestyles='dotted', label='lower limit Zenith   (scheduled start time -%s)' % int((lower_threshold_zenith - 1/120)*60*60))
        bottom_ax.vlines((time_zenith_schedule[4]+upper_threshold_zenith)*60,-5000,5000,colors='grey',linestyles='dashed', label='upper limit Zenith   (scheduled start time +%s)' % int((upper_threshold_zenith + 1/120)*60*60))
        bottom_ax.vlines((time_zenith_schedule[5]-lower_threshold_zenith)*60,-5000,5000,colors='grey',linestyles='dotted')
        bottom_ax.vlines((time_zenith_schedule[5]+upper_threshold_zenith)*60,-5000,5000,colors='grey',linestyles='dashed')
        bottom_ax.vlines((time_zenith_schedule[6]-lower_threshold_zenith)*60,-5000,5000,colors='grey',linestyles='dotted')
        bottom_ax.vlines((time_zenith_schedule[-2]+upper_threshold_zenith)*60,-5000,5000,colors='grey',linestyles='dashed')
        bottom_ax.vlines((time_zenith_schedule[-1]-lower_threshold_zenith)*60,-5000,5000,colors='grey',linestyles='dotted')
        bottom_ax.vlines((time_zenith_schedule[-1]+upper_threshold_zenith)*60,-5000,5000,colors='grey',linestyles='dashed')
        bottom_ax.grid()
        bottom_ax.set_title('%s DSCD over minutes' % date)
        bottom_ax.legend()
        bottom_ax.set_xlabel('MINUTES')
        bottom_ax.set_ylabel('DSCD')
        bottom_ax.set_xlim([0,60])
        bottom_ax.set_ylim([-100,100])
        bottom_ax.set_xticks([0,5,10,15,20,25,30,35,40,45,50,55,60])
        plt.tight_layout()
plt.show()

Pandora 34
     timeflag O4VIS
     timeflag O4UV
     timeflag O3VIS
     timeflag O3UV
     timeflag NO2VIS-SMALL
     timeflag NO2VIS
     timeflag NO2UV
     timeflag HONO
     timeflag HCHO-WIDE
     timeflag HCHO
     timeflag CHOCHO
     timeflag BrO
     timeflag BRO
Pandora 35
     timeflag O4VIS
     timeflag O4UV
     timeflag O3VIS
     timeflag O3UV
     timeflag NO2VIS-SMALL
     timeflag NO2VIS
     timeflag NO2UV
     timeflag HONO
     timeflag HCHO-WIDE
     timeflag HCHO
     timeflag CHOCHO
     timeflag BrO
     timeflag BRO
Pandora 36
     timeflag O4VIS
     timeflag O4UV
     timeflag O3VIS
     timeflag O3UV
     timeflag NO2VIS-SMALL
     timeflag NO2VIS
     timeflag NO2UV
     timeflag HONO
     timeflag HCHO-WIDE
     timeflag HCHO
     timeflag CHOCHO
     timeflag BrO
     timeflag BRO


In [16]:
for i in range(100):plt.close()